In [1]:
import torch
from img_utils import * #필자가 따로 만든 데이터 전처리 라이브러리
from tr_net_se import ImageTransformNet #트랜스포머 net 라이브러리 불러오기

from PIL import Image
import numpy as np

In [2]:
#GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 데이터셋 표준화를 위한 기본정보 # 이게 이미지 정규화임
imgNet_val = {'mean' : [0.485, 0.456, 0.406], 'std' : [0.229, 0.224, 0.225]}

In [3]:
import glob

img_dir = './test_img/' #이미지가 저장된 경로 #여기에 넣고싶은 파일 넣기
img_paths = glob.glob(img_dir + '*.jpg') + glob.glob(img_dir + '*.png')

tas_tensor_list = list()

tensor_size = (1080, 1920)
img_shape = [1080, 1920] #모든 이미지는 다 1080p임 #조정하고싶으면 마음대로

for img_path in img_paths:
    task_tensor, _ = preprocess_img(img_path, tensor_size, imgNet_val, device)
    tas_tensor_list.append(task_tensor)

#리스트 내 이미지 텐서를 배치 텐서로 결합
batch_task_tensor = torch.cat(tas_tensor_list, dim=0)
print(batch_task_tensor.size())


torch.Size([6, 3, 1080, 1920])


In [4]:
import gdown #학습시킨 가중치 파일은 구글드라이브에 있음
import os

In [5]:
# 파일 ID 추출
file_id = "1Hg9mMnH_Y9BHEtTLu7x8EupNC_VdcZt9"
# 다운로드 링크 생성
download_url = f"https://drive.google.com/uc?id={file_id}"
# 파일 다운로드
output_file = "SE4style_TR_net.pth"

# 파일 다운로드
if not os.path.exists(output_file):
    gdown.download(download_url, output_file, quiet=False)
    print("다운로드 완료")
else:
    print("파일이 이미 존재함")

파일이 이미 존재함


In [6]:
tr_net = ImageTransformNet() #모델 인스턴스화

state_dict = torch.load(output_file, weights_only=True) #weight파일 불러오기

tr_net.load_state_dict(state_dict) #불러온 weight파일을 모델에 붙이기

tr_net.to(device) #tr모델을 GPU로 이전
print()

In [7]:
tr_net.eval() #이미지를 평가모드로 전환

with torch.no_grad(): #그래디언트 추적금지
    #배치 이미지를 한번에 Style transfer
    result_tensor = tr_net(batch_task_tensor)

for i in range(result_tensor.size(0)):
    # 배치 텐서에서 개별 이미지 추출
    img_tensor = result_tensor[i].unsqueeze(0) 
    print(img_tensor.size())
    res_img = deprocess_img(img_tensor, img_shape, imgNet_val)
    #res_img는 PIL 이미지로 변환까지 수행되게 deprocess_img를 설계함
    res_img.save(f"res_{i+1:02d}.jpg") #이미지 저장

torch.Size([1, 3, 1080, 1920])
torch.Size([1, 3, 1080, 1920])
torch.Size([1, 3, 1080, 1920])
torch.Size([1, 3, 1080, 1920])
torch.Size([1, 3, 1080, 1920])
torch.Size([1, 3, 1080, 1920])
